<a href="https://colab.research.google.com/github/countifs/youtube_playlist_crawling/blob/main/%EC%9C%A0%ED%8A%9C%EB%B8%8C_%EC%9E%AC%EC%83%9D%EB%AA%A9%EB%A1%9D_%ED%81%AC%EB%A1%A4%EB%A7%81(%EA%B5%AC%EA%B8%80%EC%BD%94%EB%9E%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **유튜브 재생목록 크롤링 (구글코랩)**


In [1]:
# 모듈 설치하기
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 5.5 MB/s 
     |████████████████████████████████| 140 kB 46.3 MB/s 
     |████████████████████████████████| 384 kB 39.1 MB/s 
     |████████████████████████████████| 58 kB 3.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machin

In [3]:
# 웹 브라우저 작동을 위한 라이브러리
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys

# 웹브라우저 작동을 기다리기 위한 라이브러리
import time
import random

# 시간 관련 라이브러리
from datetime import datetime, timedelta
from pytz import timezone

# IPython
from IPython.display import display

# 경고 무시
import warnings
warnings.filterwarnings(action='ignore')

# 데이터프레임 및 CSV 파일 저장을 위한 라이브러리
import pandas as pd

# 데이터프레임 출력
from tabulate import tabulate

# 크롬드라이버 option설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def get_playlist(url, rep_text):

    # 브라우저 생성
    driver = webdriver.Chrome('chromedriver', options=options)

    # 웹사이트 열기
    driver.get(url)

    # 로딩이 끝날 때까지 2초 정도 기다림
    driver.implicitly_wait(2)

   # 안정적인 페이지 소스 추출을 위해 3초 정도 기다림
    time.sleep(3)

    # 페이지 소스 추출
    html_source = driver.page_source
    soup_source = BeautifulSoup(html_source, 'lxml')

    # 파싱정보 가져오기
    parsing = soup_source.find_all('a', class_ = 'yt-simple-endpoint style-scope ytd-playlist-video-renderer')
    video_time = soup_source.find_all('span', class_ ='style-scope ytd-thumbnail-overlay-time-status-renderer') #검색했을 때 검색숫자가 안맞아서 확인이 필요함

    # 파싱정보 정리하기
    name_list = []
    url_list = []
    time_list = []

    for i in range(len(parsing)):
        name_list.append(parsing[i].text.strip())
        main = 'https://www.youtube.com'
        sub = parsing[i].get('href')  
        url_list.append(f'{main}{sub}')
        time_list.append(video_time[i].text.strip())

   # 출력용 데이터 프레임 구성하기     
    playlist = {
        '제목' : name_list,
        '시간' : time_list,
        '확인' : '',    
        'No' : range(1, len(parsing)+1),
        'URL' : url_list, 
    }
   # 제목에서 제거할 문자 변환하기
    playlist = pd.DataFrame(playlist)
    playlist['제목'] = playlist['제목'].apply(lambda x: x.replace(rep_text,'').strip())

    return playlist

In [5]:
# url 주소 입력
url = 'https://www.youtube.com/playlist?list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS'

# 재생목록 제목에서 제거할 텍스트(문자)를 입력
rep_text  = ''

In [8]:
get_playlist(url, rep_text).style.hide_index()

제목,시간,확인,No,URL
01 빅데이터 시스템 이해 [도시빅데이터와 머신러닝],27:27,,1,https://www.youtube.com/watch?v=TnoNfaxUnpQ&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=1
03 Decision Tree (의사결정 나무) [도시빅데이터와 머신러닝],41:58,,2,https://www.youtube.com/watch?v=GzaRpXVhYuk&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=2
04-1 Ensemble Bagging Tree & Random Forest (앙상블 배깅트리와 랜덤 포레스트) [도시빅데이터와 머신러닝],28:19,,3,https://www.youtube.com/watch?v=kHPoMB8A-Kk&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=3
04-2 Ensemble Boosting ADA & GBM (부스팅기술) [도시빅데이터와 머신러닝],24:26,,4,https://www.youtube.com/watch?v=8qMavUfI0CU&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=4
05-1 Support Vector Machine (SVM hard margin) [도시빅데이터와 머신러닝],37:15,,5,https://www.youtube.com/watch?v=7Zk4BEseXqg&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=5
[Paper Review] GNN for Time Series Anomaly detection,28:16,,6,https://www.youtube.com/watch?v=n-ZDl3d7vR4&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=6
05-2 SVM Soft margin & kernel trick & 생활인구 data set [도시빅데이터와 머신러닝],38:01,,7,https://www.youtube.com/watch?v=mwtajf5X9Rg&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=7
06-1 인공신경망 (Neural Network ) [도시빅데이터와 머신러닝],40:07,,8,https://www.youtube.com/watch?v=ptlyl-FBnU4&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=8
08-2: Transformer,1:02:28,,9,https://www.youtube.com/watch?v=Yk1tV_cXMMU&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=9
07-0 Colab 연결 방법 [도시빅데이터와 머신러닝],6:29,,10,https://www.youtube.com/watch?v=dIYFpmMPOtA&list=PLmBCVfRBqN1YS-Ymle-agRHExCEebf2QS&index=10
